### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq
# Groq is an inference engine that uses LPUs for speed and capacity

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
#!pip install langchain_groq

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a10df84a740>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a10df87c940>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
#!pip install langchain_core

In [5]:
# Langchain_core
# systemmessage is how the model should work
# humanmessage is the actual query against the model

from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content='The French translation for "Hello, How are you?" is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour** means "Hello"\n* **Comment allez-vous?** means "How are you?" (formal, polite)\n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 21, 'total_tokens': 100, 'completion_time': 0.143636364, 'prompt_time': 0.00132149, 'queue_time': 0.020472307, 'total_time': 0.144957854}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1550705-cff3-48e0-9c6e-0db28ca62948-0', usage_metadata={'input_tokens': 21, 'output_tokens': 79, 'total_tokens': 100})

In [7]:
# Langchain has system output parsers
# This just strips it down to the single message

from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'The French translation for "Hello, How are you?" is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour** means "Hello"\n* **Comment allez-vous?** means "How are you?" (formal, polite)\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [8]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Salut, comment vas-tu ?** (Informal, singular)\n* **Bonjour, comment allez-vous ?** (Formal, singular)\n* **Bonjour, comment allez-vous tous ?** (Formal, plural)\n\nThe best option depends on the context and who you are speaking to. \n\n\nLet me know if you\'d like more examples or have other phrases you want translated!\n'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [10]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [13]:
#!pip install streamlit